In [536]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma
import scipy as sp

In [537]:
class OneToOneITU():
    def __init__(self, n, m, parameters , lbs=(0, 0), tol=1e-9):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
        self.tol = tol 
       
        self.A_ij = parameters[0]
        self.B_ij = parameters[1]

    def D_ij(self, U_i, V_j):
        return (U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij)/(1+ self.A_ij)

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, j_idx]

    def get_V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]

In [538]:
def transpose_mkt(self):
    n, m = self.n, self.m
    self.m = n
    self.n = m

    self.A_ij = 1 / self.A_ij.T
    self.B_ij = self.B_ij.T/ self.A_ij

OneToOneITU.transpose_mkt = transpose_mkt

In [539]:
def ITU_parallel_auction(self):

    V_j = np.ones(self.m) * self.lb_V
    mu_ij = np.ones( self.n, dtype= int) * (self.m +1)

    unassigned = np.where(mu_ij == self.m + 1)[0] 
    n_unassigned = len(unassigned)
    iter = 0

    while n_unassigned > 0:# and iter <500000:
        U_ij = self.get_U_ij( V_j, unassigned)
        # print(U_ij)
        perm_unmatched = np.all(U_ij <= self.lb_U, axis = 1)
        
        if np.any(perm_unmatched):
            mu_ij[unassigned] += (self.m - mu_ij[unassigned]) * perm_unmatched
            # print(perm_unmatched)
            # print(mu_ij)
        
        else:
            
            U_ij = np.concatenate((U_ij, self.lb_U * np.ones((n_unassigned,1))), axis = 1)
            U_ij_sorted_id = np.argsort(U_ij, axis = 1)
            j_i = U_ij_sorted_id[:,-1]
            w_i = U_ij[np.arange(n_unassigned),U_ij_sorted_id[:,-2] ]
      
            j_unique = np.unique(j_i)
            #offers = w_i[:,None] * ( j_i[:,None] == j_unique[None,:])  - 1e22 *  ( j_i[:,None] != j_unique[None,:]) 
            #print(np.shape( self.get_V_ij(unassigned[:,None],j_unique[None,:],w_i[:,None])))
            offers = (self.get_V_ij(unassigned[:,None],j_unique[None,:],w_i[:,None]) * ( j_i[:,None] == j_unique[None,:]) 
                       - 1e22 *  ( j_i[:,None] != j_unique[None,:]) )
            i_j_among_unass = np.argmax(offers, axis = 0)
            i_j = unassigned[ i_j_among_unass]
            best_offer_j = np.max(offers, axis = 0)

            # print(i_j)
            #print(offers.round(1))
            

            mu_ij[  np.any( mu_ij[:,None] == j_unique , axis = 1) ] = self.m +1
            mu_ij[i_j] = j_unique
            # print(mu_ij)
            

            #V_j[j_unique] += np.maximum(self.get_V_ij(i_j,j_unique, best_offer_j) - V_j[j_unique], self.tol)
            V_j[j_unique] += np.maximum(best_offer_j - V_j[j_unique], self.tol)
            
        unassigned = np.where(mu_ij == self.m + 1)[0]
        # print(unassigned)
        n_unassigned = len(unassigned)
        iter += 1


    matched_i =  mu_ij < self.m
    U_i = np.ones(self.n) * self.lb_U
    U_i[matched_i] = self.get_U_ij(V_j, matched_i)[np.arange(matched_i.sum()), mu_ij[matched_i]]
    mu_ij = mu_ij[:,None] == np.arange(self.m +1)
    
    

    return mu_ij, U_i, V_j


OneToOneITU.ITU_parallel_auction = ITU_parallel_auction

Generate a a parametrized example

In [588]:
np.random.seed(10)
n_i = 340
m_j = 300

θ_true =  np.array([.5,.5,5,5])
k = len(θ_true)
X_ijk = np.random.normal(1,1, size = [n_i,m_j, k])
 

A = np.exp( X_ijk[:,:,:2] @ θ_true[:2])
B = X_ijk[:,:,2:4] @ θ_true[2:4]

In [589]:
obs_mrk = OneToOneITU( n_i,m_j, (A,B), tol=0.00001)

In [590]:
mu_obs, u_true, v_true= obs_mrk.ITU_parallel_auction()

In [591]:
# obs_mrk.get_U_ij(v_true, np.arange(obs_mrk.n))

In [592]:
np.min((u_true[:,None] + obs_mrk.A_ij * v_true[None,:] - obs_mrk.B_ij) / (1+obs_mrk.A_ij))

-2.7578739956562613e-15

In [593]:
np.min(obs_mrk.D_ij(u_true, v_true) )

-2.7578739956562613e-15

In [594]:
print( np.max(mu_obs[:,-1] * ( u_true - true_mkt.lb_U)))
print( np.max((mu_obs[:,:-1].sum(0) == 0) * ( v_true - true_mkt.lb_V)))

0.0
0.0


Now let us compute the $j$-favourite solution.

In [610]:
obs_mrk.transpose_mkt()
mu_obs_i_fav, v_true_i_fav, u_true_i_fav= obs_mrk.ITU_parallel_auction()
print(np.min(obs_mrk.D_ij(v_true_i_fav, u_true_i_fav) ))
obs_mrk.transpose_mkt()

0.0


In [611]:
np.min(u_true_i_fav - u_true)

-12.821408078233908

# Estimation

In [141]:
def Q(self,θ,mu_obs,X_ijk):
    A_θ = np.exp( X_ijk[:,:,:2] @ θ[:2])
    B_θ = X_ijk[:,:,2:4] @ θ[2:4]

    mkt_θ = OneToOneITU( n_i,m_j, (A_θ,B_θ), tol= .0001)
    mu_θ, u_θ, v_θ = mkt_θ.ITU_parallel_auction()


    Q_θ = ( (mu_obs[:,:-1] * mkt_θ.D_ij(u_θ, v_θ) ).sum() + 
            (mu_obs[:,-1] * u_θ ).sum() + 
            ((mu_obs[:,:-1].sum(0) == 0) * v_θ).sum() +
            np.log( mu_θ.sum()) +
            ((v_true - v_θ)**2).sum())
    


    print(Q_θ)
    print( θ.round(3))
    return Q_θ

OneToOneITU.Q = Q

In [142]:
obs_mrk.Q(np.array([.5,.5,5,5]), mu_obs, X_ijk)

5.703782474656201
[0.5 0.5 5.  5. ]


5.703782474656201

In [143]:
def maximum_score_estimator(self,θ_0,mu_obs,X_ijk ):

    problem = sp.optimize.minimize( lambda θ: self.Q(θ,mu_obs,X_ijk)  ,θ_0)  

    return problem.x


OneToOneITU.maximum_score_estimator = maximum_score_estimator

In [144]:
obs_mrk.maximum_score_estimator( np.array([1,1,1,1]), mu_obs,X_ijk )

888.5390142595998
[1. 1. 1. 1.]
888.5390156765527
[1. 1. 1. 1.]
888.5390129608531
[1. 1. 1. 1.]
888.5390129677575
[1. 1. 1. 1.]
888.5390104595604
[1. 1. 1. 1.]
772.0132878217295
[0.678 1.295 1.293 1.862]
772.0132887419104
[0.678 1.295 1.293 1.862]
772.0132911506973
[0.678 1.295 1.293 1.862]
772.0132827084642
[0.678 1.295 1.293 1.862]
772.0132898375537
[0.678 1.295 1.293 1.862]
623.8903532215913
[0.304 1.211 2.073 2.409]
623.8903464522864
[0.304 1.211 2.073 2.409]
623.8903690432707
[0.304 1.211 2.073 2.409]
623.8903459605907
[0.304 1.211 2.073 2.409]
623.8903603065842
[0.304 1.211 2.073 2.409]
409.20333178748984
[0.056 0.867 3.021 3.283]
409.20331569133435
[0.056 0.867 3.021 3.283]
409.2033458596274
[0.056 0.867 3.021 3.283]
409.20332967970484
[0.056 0.867 3.021 3.283]
409.2033353788144
[0.056 0.867 3.021 3.283]
515.9046688590463
[0.673 1.115 5.012 4.448]
515.9046595193164
[0.673 1.115 5.012 4.448]
515.9047007474683
[0.673 1.115 5.012 4.448]
515.9046698818406
[0.673 1.115 5.012 4.448]
5

array([0.41744843, 0.50830008, 4.68633428, 4.92845468])